# Ames Housing Data

#### Imports

In [1]:
import os
import math
import warnings

In [2]:
import numpy             as np
import pandas            as pd
import matplotlib.pyplot as plt
import seaborn           as sns

In [3]:
from sklearn.preprocessing       import RobustScaler
from sklearn.model_selection     import train_test_split
from sklearn.model_selection     import GridSearchCV
from sklearn.metrics             import mean_squared_error
from xgboost                     import XGBRegressor
from xgboost                     import plot_importance
from sklearn.ensemble            import RandomForestRegressor
from sklearn.linear_model        import Lasso

In [4]:
%matplotlib inline

In [5]:
pd.set_option( 'display.max_columns', None )
sns.set( rc = { 'figure.figsize' : ( 24, 5 ) } )
warnings.filterwarnings( 'ignore' )

#### Global Functions

In [6]:
def get_numerical_features( df ):
    return df.select_dtypes( include = [ 'int64', 'float64' ] ).columns

def get_categorical_features( df ):
    return df.select_dtypes( include = [ 'object' ] ).columns

def return_features_with_null( df ):
    still_missing = pd.DataFrame( len( df[ get_categorical_features ] ) - df[ get_categorical_features ].count() )
    return pd.DataFrame( still_missing[ still_missing[ 0 ] > 0 ] )

def return_rows_with_null( df ):
    null_columns = df.columns[ df.isnull().any() ]
    print( pd.DataFrame( df[ df.isnull().any( axis = 1 ) ][ null_columns ].head( 10 ) ) )

def na_heatmap( df ):
    df      = df[ sorted( df.columns ) ]
    fig, ax = plt.subplots( figsize = ( 25, 5 ) )
    sns.heatmap( df.isnull(), yticklabels = False, cbar = False )
    
def get_facet_grid( df, feature_type ):
    if feature_type == 'numerical':
        facet_grid = pd.melt( df, value_vars = sorted( get_numerical_features( df ) ) )
        grid_plot  = sns.FacetGrid( facet_grid, col = 'variable', col_wrap = 10, sharex = False, sharey = False)
        grid_plot  = grid_plot.map( sns.distplot, 'value' )
        
        return grid_plot
    
    elif feature_type == 'categorical':
        facet_grid = pd.melt( df, value_vars = sorted( get_categorical_features( df ) ) )
        grid_plot  = sns.FacetGrid( facet_grid, col = 'variable', col_wrap = 10, sharex = False, sharey = False )
        grid_plot  = grid_plot.map( sns.countplot, 'value' )

        plt.xticks( rotation = 'vertical' )
        [ plt.setp( ax.get_xticklabels(), rotation = 60 ) for ax in grid_plot.axes.flat ]
        grid_plot.fig.tight_layout()
        
        return grid_plot
    
    else:
        print( 'Please specify "numerical" of "categorical"' )

#### Load Data

In [7]:
data_path = os.getcwd() + '\\..\\..\\..\\data\\ames_housing\\'
train_raw = pd.read_csv( data_path + 'train.csv' )
test_raw  = pd.read_csv( data_path + 'test.csv' )

train_X = train_raw.drop( [ 'SalePrice', 'Id' ], axis = 1 )
train_y = train_raw[ 'SalePrice' ]
test_X  = test_raw.drop( [ 'Id' ], axis = 1 )

#### Explore the Data

In [8]:
# print( "Train: {} \nTest: {}".format( train_X.shape, test_X.shape ) )

In [9]:
# train_X.head()

In [10]:
# train_X.describe()

In [11]:
full_X    = pd.concat( [train_X, test_X] )
train_end = len( train_X )
test_end  = len( full_X )

# print( full_X.shape )

In [12]:
# get_facet_grid( full_X, 'numerical' )

In [13]:
# get_facet_grid( full_X, 'categorical' )

In [14]:
# na_heatmap( train_X )

In [15]:
train_y = np.log1p( train_y )
# sns.distplot( train_y, bins = 75 )

#### Data Preprocessing

###### Make Ranked Features

In [16]:
# Ranking from Kaggle data info
def convert_to_ranked( df ):
    df[ 'Alley'        ].replace( { 'Grvl' : 1, 'Pave' : 2 }, inplace = True )
    df[ 'BsmtCond'     ].replace( { 'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4 }, inplace = True )
    df[ 'BsmtExposure' ].replace( { 'No' : 1, 'Mn' : 2, 'Av' : 3, 'Gd' : 4 }, inplace = True )
    df[ 'BsmtFinType1' ].replace( { 'Unf' : 1, 'LwQ' : 2, 'Rec' : 3, 'BLQ' : 4, 'ALQ' : 5, 'GLQ' : 6 }, inplace = True )
    df[ 'BsmtFinType2' ].replace( { 'Unf' : 1, 'LwQ' : 2, 'Rec' : 3, 'BLQ' : 4, 'ALQ' : 5, 'GLQ' : 6 }, inplace = True )
    df[ 'BsmtQual'     ].replace( { 'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5 }, inplace = True )
    df[ 'ExterCond'    ].replace( { 'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5 }, inplace = True )
    df[ 'ExterQual'    ].replace( { 'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5 }, inplace = True )
    df[ 'Fence'        ].replace( { 'MnWw' : 1, 'GdWo' : 2, 'MnPrv' : 3, 'GdPrv' : 4 }, inplace = True )
    df[ 'FireplaceQu'  ].replace( { 'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5 }, inplace = True )
    df[ 'Functional'   ].replace( { 'Sal' : 1, 'Sev' : 2, 'Maj2' : 3, 'Maj1' : 4, 'Mod' : 5, 'Min2' : 6, 'Min1' : 7, 'Typ' : 8 }, inplace = True )
    df[ 'GarageCond'   ].replace( { 'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5 }, inplace = True )
    df[ 'GarageFinish' ].replace( { 'Unf' : 1, 'RFn' : 2, 'Fin' : 3 }, inplace = True )
    df[ 'GarageQual'   ].replace( { 'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5 }, inplace = True )
    df[ 'HeatingQC'    ].replace( { 'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5 }, inplace = True )
    df[ 'KitchenQual'  ].replace( { 'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5 }, inplace = True )
    df[ 'LandSlope'    ].replace( { 'Sev' : 1, 'Mod' : 2, 'Gtl' : 3 }, inplace = True )
    df[ 'LandContour'  ].replace( { 'Low' : 1, 'HLS' : 2, 'Bnk' : 3, 'Lvl' : 4 }, inplace = True )
    df[ 'LotShape'     ].replace( { 'Reg' : 1, 'IR1' : 2, 'IR2' : 3, 'IR3' : 4 }, inplace = True )
    df[ 'PoolQC'       ].replace( { 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5 }, inplace = True )
    df[ 'PavedDrive'   ].replace( { 'N' : 1, 'P' : 2, 'Y' : 3 }, inplace = True )
    df[ 'Utilities'    ].replace( { 'ELO' : 1, 'NoSeWa' : 2, 'NoSewr' : 3, 'AllPub' : 4 }, inplace = True )

In [17]:
convert_to_ranked( full_X )

###### Create Features

In [18]:
full_X[ 'TotalLivAreaSF' ] = full_X[ '1stFlrSF'     ] + full_X[ '2ndFlrSF' ]
full_X[ 'TotalOtherSF' ]   = full_X[ 'TotalBsmtSF'  ] + full_X[ 'GrLivArea' ] 
full_X[ 'TotalBath'   ]    = full_X[ 'BsmtFullBath' ] + ( 0.5 * full_X[ 'BsmtHalfBath' ] ) + full_X[ 'FullBath' ] + ( 0.5 * full_X[ 'HalfBath' ] )
full_X[ 'TotalPorchSF' ]   = full_X[ 'OpenPorchSF'  ] + full_X[ 'EnclosedPorch' ] + full_X[ '3SsnPorch' ] + full_X[ 'ScreenPorch' ]

###### Make Categorical

In [19]:
def convert_to_category( df ):
    df[ 'MSSubClass' ].replace( { 20 : '20', 30 : '30', 40 : '40', 45 : '45', 50 : '50', 60 : '60',
                                  70 : '70', 75 : '75', 80 : '80', 85 : '85', 90 : '90', 120 : '120',
                                  150 : '150', 160 : '160', 180 : '180', 190 : '190' }, inplace = True )
    
    df[ 'MoSold' ].replace( { 1 : 'Jan', 2 : 'Feb', 3 : 'Mar', 4 : 'Apr', 5 : 'May', 6 : 'Jun', 
                              7 : 'Jul', 8 : 'Aug', 9 : 'Sep', 10 : 'Oct', 11 : 'Nov', 12 : 'Dec' }, inplace = True )
    
    df[ 'YrSold' ].replace( { 2006 : '2006', 2007 : '2007', 2008 : '2008', 2009 : '2009', 2010 : '2010'  }, inplace = True )

In [20]:
convert_to_category( full_X )

###### Replace Missing Values

In [21]:
# na_heatmap( full_X )

In [22]:
fill_with_none = [ 'Alley', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'BsmtQual', 'Fence', 'FireplaceQu', 
                   'Functional', 'GarageCond', 'GarageFinish', 'GarageQual', 'GarageType', 'KitchenQual', 'MasVnrType', 'MiscFeature', 'PoolQC' ]

fill_with_zero = [ 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath', 'BsmtUnfSF', 'GarageArea', 'GarageCars', 'TotalBsmtSF', 'GarageYrBlt', 'MasVnrArea', 'TotalBath', 'TotalOtherSF' ]

full_X[ fill_with_none ] = full_X[ fill_with_none ].fillna( 'None' )
full_X[ fill_with_zero ] = full_X[ fill_with_zero ].fillna( 0 )

In [23]:
train_X = full_X[ 0:train_end ]
test_X  = full_X[ train_end:test_end ]

# print( "Train: {} \nTest: {}".format( train_X.shape, test_X.shape ) )

###### Fill Value with Mode

In [24]:
train_X[ 'Utilities' ].fillna( train_X[ 'Utilities' ].mode()[0], inplace = True )
test_X[ 'Utilities' ].fillna( test_X[ 'Utilities' ].mode()[0], inplace = True )

###### Fill with Median

In [25]:
train_X[ 'LotFrontage' ] = train_X.groupby( 'Neighborhood' )[ 'LotFrontage' ].transform( lambda x: x.fillna( x.median() ) )
test_X[ 'LotFrontage' ] = test_X.groupby( 'Neighborhood' )[ 'LotFrontage' ].transform( lambda x: x.fillna( x.median() ) )

In [26]:
# return_features_with_null( train_X )

In [27]:
# return_features_with_null( test_X )

In [28]:
# return_rows_with_null( full_X )

###### Fill Remaining Missing Values

In [29]:
missing_train = [ 'Electrical' ]
missing_test  = [ 'MSZoning', 'Exterior1st', 'Exterior2nd', 'SaleType' ] 

In [30]:
train_X[ missing_train ] = train_X[ missing_train ].fillna( train_X.mode().iloc[0] )
test_X[ missing_test ]   = test_X[ missing_test ].fillna( test_X.mode().iloc[0] )

In [31]:
full_X = pd.concat( [train_X, test_X] )

###### Fix Skew

In [32]:
# get_facet_grid( full_X, 'numerical' )

In [33]:
def fix_pos_skew( feature ):
    return np.log1p( full_X[ feature ] + 1 )

In [34]:
features_to_log = [ '1stFlrSF', 'BsmtUnfSF', 'GarageArea', 'GrLivArea', 'LotArea', 'LotFrontage', 'MiscVal', 'TotalBsmtSF', 'TotalOtherSF', 'TotalLivAreaSF' ]

In [35]:
for feature in features_to_log:
    full_X[ feature ] = fix_pos_skew( feature )

In [36]:
# get_facet_grid( full_X[ features_to_log ], 'numerical' )

###### Handle Incorrect Data Input (Test Garage Year Built)

In [37]:
# print( max( full_X[ 'GarageYrBlt' ] ) )
# print( sorted( full_X[ 'GarageYrBlt' ], reverse = True )[ 0:5 ] )

In [38]:
target_index                              = full_X.loc[ full_X[ 'GarageYrBlt' ] == 2207.0 ].index[0]
full_X.loc[ target_index, 'GarageYrBlt' ] = round( np.mean( full_X[ 'GarageYrBlt' ] ) )

In [39]:
# print( max( full_X[ 'GarageYrBlt' ] ) )

###### Create One-Hot-Encoded

In [40]:
count_data                  = [ 'BedroomAbvGr', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'TotalBath', 'Fireplaces', 'GarageCars', 'KitchenAbvGr', 'TotRmsAbvGrd' ]
numerical_features          = list( get_numerical_features( full_X ) )
numerical_features_to_scale = [ x for x in numerical_features if x not in count_data ]
categorical_features        = list( get_categorical_features( full_X ) )

In [41]:
full_X = pd.get_dummies( full_X, drop_first = True, prefix = categorical_features, columns = categorical_features )

###### Split Back to Train/Test

In [42]:
train_X = pd.DataFrame( full_X[ 0:train_end ] )
test_X  = pd.DataFrame( full_X[ train_end:test_end ] )

# print( "Train: {} \nTest: {}".format( train_X.shape, test_X.shape ) )

#### Standardize Features

In [43]:
scaler = RobustScaler()

train_X[ numerical_features_to_scale ] = scaler.fit_transform( train_X[ numerical_features_to_scale ], train_y ) 
test_X[ numerical_features_to_scale ]  = scaler.transform( test_X[ numerical_features_to_scale ] )

In [44]:
# train_X.head()

#### Build & Compare Models

In [45]:
X_train, X_test, y_train, y_test = train_test_split( train_X, train_y, test_size = 0.33, random_state = 42 )
# # print( "Train: {} \nTest: {}".format( X_train.shape, X_test.shape ) )

In [46]:
rmse_scores = { 'XGBoost'     : 0,
                'XGBoostSlim' : 0,
                'Lasso'       : 0 }

###### XGBoost

In [47]:
# These are the params that ended up being th best after an initial grid search
param_grid = { 'learning_rate'    : [ 0.1 ],
               'max_depth'        : [ 4 ],
               'subsample'        : [ 0.75 ],
               'colsample_bytree' : [ 1 ],
               'n_estimators'     : [ 100 ],
               'reg_alpha'        : [ 0 ],
               'reg_lambda'       : [ 0.25 ] }

xgbm = GridSearchCV( XGBRegressor(), cv = 5, param_grid = param_grid, n_jobs = -1, scoring = 'neg_mean_squared_error', verbose = 1 )
xgbm.fit( X_train, y_train )

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    4.3s remaining:    6.5s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    4.3s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'learning_rate': [0.1], 'max_depth': [4], 'subsample': [0.75], 'colsample_bytree': [1], 'n_estimators': [100], 'reg_alpha': [0], 'reg_lambda': [0.25]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=1)

In [48]:
y_pred = xgbm.predict( X_test )
rmse_scores[ 'XGBoost' ] = math.sqrt( mean_squared_error( y_test, y_pred ) )

###### XGBoost Slim

In [49]:
# These are the params that ended up being th best after an initial grid search
xgb = XGBRegressor( learning_rate    =  0.1,
                    max_depth        =  4,
                    subsample        =  0.75,
                    colsample_bytree =  1,
                    n_estimators     =  100,
                    reg_alpha        =  0,
                    reg_lambda       =  0.25 )

xgb.fit( train_X, train_y )

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=0.25, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.75)

In [50]:
result             = pd.DataFrame( { 'Feature' : X_train.columns, 'Importance' : xgb.feature_importances_ } )
result             = result.sort_values( by = [ 'Importance' ], ascending = False ).reset_index( drop = True )
important_features = list( result[ result[ 'Importance' ] > 0.004 ][ 'Feature' ] )

In [51]:
len( X_train.columns )

278

In [52]:
X_train_copy = X_train[ important_features ].copy()
X_test_copy  = X_test[ important_features ].copy()

In [53]:
len( X_train_copy.columns )

48

In [54]:
xgb = XGBRegressor( learning_rate    =  0.1,
                    max_depth        =  4,
                    subsample        =  0.5,
                    colsample_bytree =  0.5,
                    n_estimators     =  200,
                    reg_alpha        =  0,
                    reg_lambda       =  0.25 )

xgb.fit( X_train_copy, y_train )

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.5, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=200,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=0.25, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.5)

In [55]:
y_pred = xgb.predict( X_test_copy )
rmse_scores[ 'XGBoostSlim' ] = math.sqrt( mean_squared_error( y_test, y_pred ) )

###### Lasso

In [56]:
# These are the params that ended up being th best after an initial grid search
param_grid = { 'alpha'    : [ 0.001 ], 
               'max_iter' : [ 750 ] }

lasso = GridSearchCV( Lasso(), cv = 5, param_grid = param_grid, n_jobs = 6, scoring = 'neg_mean_squared_error', verbose = 1 )
lasso.fit( X_train, y_train )

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   5 out of   5 | elapsed:    1.1s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid='warn', n_jobs=6,
       param_grid={'alpha': [0.001], 'max_iter': [750]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=1)

In [57]:
y_pred = lasso.predict( X_test )
rmse_scores[ 'Lasso' ] = math.sqrt( mean_squared_error( y_test, y_pred ) )

#### Review RMSE Scores

In [58]:
pd.DataFrame( rmse_scores, columns = list( rmse_scores.keys() ), index = [ 1 ] )

,XGBoost,XGBoostSlim,Lasso
1,0.139345,0.130509,0.129009


#### Run Selected Model on Data

In [59]:
lasso.best_params_

{'alpha': 0.001, 'max_iter': 750}

In [60]:
# These are the params that ended up being th best after an initial grid search
param_grid = { 'alpha'    : [ 0.001 ], 
               'max_iter' : [ 5000 ] }

lasso = GridSearchCV( Lasso(), cv = 10, param_grid = param_grid, n_jobs = 6, scoring = 'neg_mean_squared_error', verbose = 1 )
lasso.fit( train_X, train_y )

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=6)]: Done  10 out of  10 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=6)]: Done  10 out of  10 | elapsed:    0.5s finished


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid='warn', n_jobs=6,
       param_grid={'alpha': [0.001], 'max_iter': [5000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=1)

In [61]:
predictions = np.exp( lasso.predict( test_X ) )

#### Create Submission

In [62]:
submission = pd.DataFrame(
    { 'Id'        : test_raw[ 'Id' ],
      'SalePrice' : predictions } 
)

submission.to_csv( '.\\ames_housing_submit.csv', index = False )
submission.head( 5 )

,Id,SalePrice
0,1461,110839.353775
1,1462,154453.949230
2,1463,183257.659738
3,1464,204060.062722
4,1465,195683.528401
